In [1]:
#!/usr/bin/python


from __future__ import print_function
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from scipy.optimize import NonlinearConstraint, LinearConstraint
from scipy.optimize import BFGS, minimize, Bounds, SR1
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN
import numpy
from keras.models import model_from_json, load_model
from pathlib import Path
import os.path
import time
import os
import math
import pyipopt
from numpy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import json
from pickle import load
scaler_X = load(open('scaler_X.pkl', 'rb'))
scaler_y = load(open('scaler_y.pkl', 'rb'))

print(scaler_X.mean_)
print(scaler_y.mean_)
print(scaler_X.var_)
print(scaler_y.var_)
from Autoencoderclasses import Encoder, Decoder, DenseEncoder, DenseDecoder 
from firstprinciples import diff_schemes 

dim_x = 45
dim_t = 400

t_step = 10
dim_x_red = 6
dim_u = 2

delta=0.8
hc=delta/dim_t #delta/100
init_value = 7.5
realtime_data = np.array([init_value]*(dim_x + 1))
realtime_data[0] = realtime_data[-1] = 0
xi_record = realtime_data
u1_record=[]
u2_record=[]
rfd_record = realtime_data.reshape(1,(dim_x+1))
NUM_HORIZON = 2
NUM_MPC_INPUTS = 2 * NUM_HORIZON 
NUM_MPC_CONSTRAINTS=NUM_HORIZON
NUM_MPC_ITERATION = 10


/home/wulab/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-04-12 15:01:19.667871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 15:01:19.783111: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 15:01:19.786421: W tensorflow/compiler/xla/stream_executor/platform/default

[0.         0.41308978 0.69049989 0.93965885 1.1505734  1.31712663
 1.43593837 1.50574973 1.52625246 1.49784779 1.42129797 1.29812005
 1.13108851 0.92575573 0.72862593 0.68780734 0.7323762  0.95539395
 1.18262314 1.37097919 1.51493613 1.61185639 1.66050622 1.66043322
 1.61163235 1.5145456  1.37039514 1.18180629 0.95429469 0.73099307
 0.68635883 0.72723764 0.92451469 1.12997335 1.29709011 1.42032648
 1.49692297 1.52537487 1.50492873 1.43518891 1.31646646 1.15002058
 0.93922963 0.69020705 0.41294157 0.        ]
[0.         0.41308978 0.69049989 0.93965885 1.1505734  1.31712663
 1.43593837 1.50574973 1.52625246 1.49784779 1.42129797 1.29812005
 1.13108851 0.92575573 0.72862593 0.68780734 0.7323762  0.95539395
 1.18262314 1.37097919 1.51493613 1.61185639 1.66050622 1.66043322
 1.61163235 1.5145456  1.37039514 1.18180629 0.95429469 0.73099307
 0.68635883 0.72723764 0.92451469 1.12997335 1.29709011 1.42032648
 1.49692297 1.52537487 1.50492873 1.43518891 1.31646646 1.15002058
 0.93922963 0.69

/home/wulab/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
def my_ens_prediction(num_horizon,my_rawdata,my_inputs):
    assert num_horizon == len(my_inputs) / 2
    ensemble_output = numpy.zeros((num_horizon,t_step,dim_x_red))
    my_inputs_normalized = np.array(my_inputs) / 100.0
    curr_state = encoder.call(scaler_X.transform(my_rawdata.copy().reshape((1,dim_x + 1))))

    for i in range(num_horizon):
        input = np.concatenate((curr_state,  my_inputs_normalized[2*i:(2*i+2)]), axis=None)
        rnn_input = np.tile(input,(t_step,1)).reshape((1,t_step,len(input)))
        ensemble_output[i] =rnn_model.predict(rnn_input,verbose=0)
        curr_state = ensemble_output[i,-1].reshape((1,dim_x_red))
    
    return ensemble_output




In [3]:
dir_name = os.getcwd()
test = os.listdir(dir_name)


init_value = 7.5
realtime_data = np.array([init_value]*(dim_x + 1))
realtime_data[0] = realtime_data[-1] = 0
xi_record = realtime_data
u1_record=[]
u2_record=[]
rfd_record = realtime_data.reshape(1,(dim_x+1))


for item in test:
    if item.endswith(".txt"):
        os.remove(os.path.join(dir_name, item))

nnzj = NUM_MPC_CONSTRAINTS*NUM_MPC_INPUTS
nnzh = NUM_MPC_INPUTS**2
nvar = NUM_MPC_INPUTS
x_lower=[0]* nvar
x_upper=[0]* nvar
for i in range(int(NUM_MPC_INPUTS/2)):
    x_lower[2*i]=-200.0
    x_lower[2 * i+1] = -200.0
    x_upper[2 * i] = 0.0
    x_upper[2 * i + 1] = 0.0
x_L = array(x_lower)
x_U = array(x_upper) 
ncon = NUM_MPC_CONSTRAINTS
g_L = array([-2e19]*NUM_HORIZON)
g_U = array([0]*NUM_HORIZON)



title_array1 = ['Dense 300 units 6 rd 1.0 noise']
title_array2 = [ 'Spectral' + s for s in title_array1]
title_array = title_array1 + title_array2

results_dict = dict()
np.random.seed(123)
noises = np.random.normal(0,1.0,(NUM_MPC_ITERATION, 1,46))
print(noises[0])

for title in title_array:
    print("Now doing: ", title)
    if title[0] == 'S':
        encoder_decoder = tf.keras.models.load_model(title + ".h5", custom_objects={'Encoder' : Encoder, 'Decoder' : Decoder})
    else:
        encoder_decoder = tf.keras.models.load_model(title + ".h5", custom_objects={'DenseEncoder' : DenseEncoder, 'DenseDecoder' : DenseDecoder})

    encoder = encoder_decoder.layers[0]
    decoder = encoder_decoder.layers[1]
    RNN_model = tf.keras.models.load_model('RNN for ' + title + '.h5', custom_objects={'Encoder' : Encoder, 'Decoder' : Decoder})

    rnn_model = tf.keras.models.load_model('RNN for ' + title + '.h5')

    def my_ens_prediction(num_horizon,my_rawdata,my_inputs):
        assert num_horizon == len(my_inputs) / 2
        ensemble_output = numpy.zeros((num_horizon,t_step,dim_x_red))
        my_inputs_normalized = np.array(my_inputs) / 100.0
        curr_state = encoder.call(scaler_X.transform(my_rawdata.copy().reshape((1,dim_x + 1))))

        for i in range(num_horizon):
            input = np.concatenate((curr_state,  my_inputs_normalized[2*i:(2*i+2)]), axis=None)
            rnn_input = np.tile(input,(t_step,1)).reshape((1,t_step,len(input)))
            ensemble_output[i] =rnn_model.predict(rnn_input,verbose=0)
            curr_state = ensemble_output[i,-1].reshape((1,dim_x_red))
        
        return ensemble_output


    def eval_f(x):
        assert len(x) == int(NUM_MPC_INPUTS)
        df_ensemble_output = my_ens_prediction(num_horizon=int(NUM_MPC_INPUTS/2),my_rawdata=realtime_data,my_inputs=x)
        offset=0
        SUM = np.sum(realtime_data**2)
        setpoint = np.zeros((1, dim_x + 1 ))
        for j in range (int(NUM_MPC_INPUTS/2)):
            est_outlet_product = scaler_y.inverse_transform(decoder.call(df_ensemble_output[j, :, :]))

            for i in range(int(t_step)):  
                for k in range(dim_x+1):
                    offset = offset + (setpoint[0,k] - (est_outlet_product[i, k])) ** 2.0*1
            if SUM > 5:
                offset = offset+10*x[2*j] **2*1e-4 + 10* x[2*j+1] **2*1e-4
            else:
                offset = offset+10*x[2*j] **2*1e-2 + 10* x[2*j+1] **2*1e-2

        return float(offset/100)

    def eval_grad_f(x):
        assert len(x) == int(NUM_MPC_INPUTS)
        step = 0.1 # we just have a small step
        objp=objm=0
        grad_f = [0.0]*NUM_MPC_INPUTS
        xpstep = [0.0]*NUM_MPC_INPUTS
        xmstep = [0.0]*NUM_MPC_INPUTS
        for i_mpc_input in range(NUM_MPC_INPUTS):
            xpstep=x.copy()
            xmstep=x.copy()
            # for each variables, we need to evaluate the derivative of the function with respect to that variable, This is why we have the for loop
            xpstep[i_mpc_input]  = xpstep[i_mpc_input]+step 
            xmstep[i_mpc_input] = xmstep[i_mpc_input]-step
            objp=eval_f(xpstep) # This function returns the value of the objective function evaluated with the variable x[i] is perturebed +step
            objm=eval_f(xmstep) # This function returns the value of the objective function evaluated with the variable x[i] is perturebed -step
            grad_f[i_mpc_input] = (objp - objm) / (2 * step) # This evaluates the gradient of the objetive function with repect to the optimization variable x[i]
        return array(grad_f)



    def apply_new(x):
        return True
    def print_variable(variable_name, value):
        for i in range(len(value)):
            print("{} {}".format(variable_name + "["+str(i)+"] =", value[i]))

    def eval_g(x):
        assert len(x) == int(NUM_MPC_INPUTS)
        g=array([-5.0]*NUM_MPC_CONSTRAINTS)
        return  g

    nnzj = NUM_MPC_CONSTRAINTS*NUM_MPC_INPUTS


    def eval_jac_g(x, flag):
        if flag:
            list_x = []
            list_y=[]
            for i in range(int(NUM_MPC_INPUTS / 2)):
                list_x = list_x + [i] * NUM_MPC_INPUTS
                list_y = list_y +list(range(0, int(NUM_MPC_INPUTS)))
            return (array(list_x),
                    array(list_y))

        else:
            assert len(x) == int(NUM_MPC_INPUTS)
            step = 0.1 
            gp=gm=numpy.zeros(NUM_MPC_CONSTRAINTS)
            xpstep=xmstep=numpy.zeros(NUM_MPC_INPUTS)
            jac_g = [[0]*int(NUM_MPC_INPUTS) for _ in range(NUM_MPC_CONSTRAINTS)]
            for i_mpc_input in range(NUM_MPC_INPUTS):
                xpstep=x.copy()
                xmstep=x.copy()
                xpstep[i_mpc_input] += step 
                xmstep[i_mpc_input] -= step
                gp=eval_g(xpstep)
                gm=eval_g(xmstep)
                for num_constraint in range(NUM_MPC_CONSTRAINTS):
                    jac_g[num_constraint][i_mpc_input] = (gp[num_constraint] - gm[num_constraint]) / (2 * step)
            return array(jac_g)

    dir_name = os.getcwd()
    test = os.listdir(dir_name)


    init_value = 7.5
    realtime_data = np.array([init_value]*(dim_x + 1))
    realtime_data[0] = realtime_data[-1] = 0
    xi_record = [realtime_data]
    u1_record=[]
    u2_record=[]
    rfd_record = realtime_data.reshape(1,(dim_x+1))


    for item in test:
        if item.endswith(".txt"):
            os.remove(os.path.join(dir_name, item))

    nnzj = NUM_MPC_CONSTRAINTS*NUM_MPC_INPUTS
    nnzh = NUM_MPC_INPUTS**2
    nvar = NUM_MPC_INPUTS
    x_lower=[0]* nvar
    x_upper=[0]* nvar
    for i in range(int(NUM_MPC_INPUTS/2)):
        x_lower[2*i]=-200.0
        x_lower[2 * i+1] = -200.0
        x_upper[2 * i] = 0.0
        x_upper[2 * i + 1] = 0.0
    x_L = array(x_lower)
    x_U = array(x_upper) 
    ncon = NUM_MPC_CONSTRAINTS
    g_L = array([-2e19]*NUM_HORIZON)
    g_U = array([0]*NUM_HORIZON)




    for main_iteration in range(NUM_MPC_ITERATION):
        state = scaler_X.transform(xi_record[-1].reshape((1,46)))
        noisy_state  = scaler_X.inverse_transform(state + noises[main_iteration])
        noisy_state2 =  noisy_state[0]
        for i in range(len(noisy_state2)):
            noisy_state2[i] = max(0, noisy_state2[i])
        realtime_data = noisy_state2
        print ("Num Iteratin: ", main_iteration)
        nlp = pyipopt.create(nvar, x_L, x_U, ncon, g_L, g_U, nnzj, nnzh, eval_f, eval_grad_f, eval_g, eval_jac_g)
        if main_iteration == 0 :
            x0 = np.array([-200.00000,-200.0, -200.00000, -200.0])
        else:
            x0=x
            x0[0:-2]=x[2:]
            x0[-2:]=x[-2:]#[0, 0]
            x_record=x
        nlp.num_option('tol', 1e-3)
        nlp.int_option('print_level', 2)
        print("Going to call solve")
        print("input_0  = {}".format(x0))
        x, zl, zu, constraint_multipliers, obj, status = nlp.solve(x0)
        nlp.close()

        print("Solution of the primal variables,input_opt")
        print("input_opt", x)


        print("Objective value")
        print("loss(x*) = {}".format(obj))
        print ("Control action=:  ", x[0], x[1])
        print("Status : " + str(status))


        dx_step = 45
        dt_step = 100 

        dt = 0.2/dt_step
        dx = np.pi/dx_step
        a = dt/dx**2
        X_array = np.linspace(0,np.pi,(dx_step+1))
        X_array = np.array(X_array)
        T_array = np.linspace(0,0.8,dt_step+1)
        initial_st = xi_record[-1]
        u1=x[0]
        u2=x[1]
        res = diff_schemes(dt,dx,a,X_array,T_array,u1,u2,initial_st)
        ic = res.init_condition()
        rfd = res.forward_diff_scheme()
        xi_record = np.vstack((xi_record, rfd))
        u1_record.append(x[0])
        u2_record.append(x[1])
        print("Current State: " , xi_record[-1])
        
    
    MPC_results_dict = {'u1' : u1_record, 'u2' : u2_record, 'state' : xi_record.tolist()}

    with open( title + ' Closed-Loop Simulation Results.json', 'w') as fp:
        json.dump(MPC_results_dict,fp)






[[-1.0856306   0.99734545  0.2829785  -1.50629471 -0.57860025  1.65143654
  -2.42667924 -0.42891263  1.26593626 -0.8667404  -0.67888615 -0.09470897
   1.49138963 -0.638902   -0.44398196 -0.43435128  2.20593008  2.18678609
   1.0040539   0.3861864   0.73736858  1.49073203 -0.93583387  1.17582904
  -1.25388067 -0.6377515   0.9071052  -1.4286807  -0.14006872 -0.8617549
  -0.25561937 -2.79858911 -1.7715331  -0.69987723  0.92746243 -0.17363568
   0.00284592  0.68822271 -0.87953634  0.28362732 -0.80536652 -1.72766949
  -0.39089979  0.57380586  0.33858905 -0.01183049]]
Now doing:  Dense 300 units 6 rd 1.0 noise


2023-04-12 15:01:21.340081: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 15:01:21.340238: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-12 15:01:21.340271: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-12 15:01:21.340297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-12 15:01:21.341555: W tensorflow/c

Num Iteratin:  0
[PyIPOPT] Ipopt will use Hessian approximation.

[PyIPOPT] Problem created
Going to call solve
input_0  = [-200. -200. -200. -200.]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Solution of the primal variables,input_opt
input_opt [-155.70764051 -200.000002    -59.44182559  -89.05146046]
Objective value
loss(x*) = 47.85751362737508
Control action=:   -155.70764051382474 -200.00000199955846
Status : 0
Current State:  [0.         0.5311099  1.03711738 1.49065106 1.867267   2.14750162
 2.31744851 2.3688249  2.29893006 2.11064237 1.81248919 1.41874385
 0.94937467 0.43011259 0.         0.         0.         0.32389039
 0.73799878 1.1053979